In [48]:
import pandas as pd

In [49]:
df_main = pd.read_csv("../dataframes/incidenti_tfl_2019.csv")
df_ages = pd.read_csv("../dataframes/age_range_counts_to_merge.csv")
df_tod = pd.read_csv("../dataframes/tod_analysis.csv")
df_vehicles = pd.read_csv("../dataframes/vehicles_by_cat.csv")


In [50]:
df_main.head()
# df_main.keys()

,$type,id,lat,lon,location,date,severity,borough,casualties,vehicles
0,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,345979,51.570865,-0.231959,On Edgware Road Near The Junction With north C...,2019-01-04T21:22:00Z,Slight,Barnet,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
1,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,345980,51.603859,-0.187240,On Willow Way Near The Junction With Long Lane,2019-01-04T23:33:00Z,Slight,Barnet,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
2,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,345981,51.512198,-0.153122,On north Audley Street Near The Junction With ...,2019-01-04T22:15:00Z,Slight,City of Westminster,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
3,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,345982,51.431480,-0.016083,On Bromley Road Near The Junction With Daneswo...,2019-01-04T18:00:00Z,Slight,Lewisham,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
4,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,345983,51.473487,0.145202,On Belmont Road Near The Junction With Bedonwe...,2019-01-04T20:45:00Z,Slight,Bexley,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...


In [51]:
# Dropped unecessary cols
# General idea keep: [id, lat, lon, date, TOD, severity, boroughs..., casualties_range..., num_vehicles...]
df_main.drop(columns=['$type', 'location', 'casualties', 'vehicles'], inplace=True)

In [52]:
# converting 'severity' to cardinal indicator
df_main['severity'] = df_main['severity'].map({'Slight': 1, 'Serious': 2, 'Fatal': 3})

In [53]:
# Encode boroughs to OHE
borough_encoded = pd.get_dummies(df_main['borough'], prefix='borough', dtype=int)
df_main = pd.concat([df_main, borough_encoded], axis=1)
df_main.drop('borough', axis=1, inplace=True)


In [55]:
df_merged = df_main.merge(df_ages, on='id', how='left')
df_merged = df_merged.merge(df_tod, on='id', how='left')
df_merged = df_merged.merge(df_vehicles, on='id', how='left')
print(df_merged.keys())
df_merged.head()

Index(['id', 'lat', 'lon', 'date', 'severity', 'borough_Barking and Dagenham',
       'borough_Barnet', 'borough_Bexley', 'borough_Brent', 'borough_Bromley',
       'borough_Camden', 'borough_City of London',
       'borough_City of Westminster', 'borough_Croydon', 'borough_Ealing',
       'borough_Enfield', 'borough_Greenwich', 'borough_Hackney',
       'borough_Hammersmith and Fulham', 'borough_Haringey', 'borough_Harrow',
       'borough_Havering', 'borough_Hillingdon', 'borough_Hounslow',
       'borough_Islington', 'borough_Kensington and Chelsea',
       'borough_Kingston', 'borough_Lambeth', 'borough_Lewisham',
       'borough_Merton', 'borough_Newham', 'borough_Redbridge',
       'borough_Richmond upon Thames', 'borough_Southwark', 'borough_Sutton',
       'borough_Tower Hamlets', 'borough_Waltham Forest', 'borough_Wandsworth',
       'casualty_age_0-23', 'casualty_age_24-30', 'casualty_age_31-38',
       'casualty_age_39-50', 'casualty_age_50+', 'time_of_day', 'Car',
       'H

,id,lat,lon,date,severity,borough_Barking and Dagenham,borough_Barnet,borough_Bexley,borough_Brent,borough_Bromley,...,casualty_age_24-30,casualty_age_31-38,casualty_age_39-50,casualty_age_50+,time_of_day,Car,Heavy_Vehicles,Motorcycle,Other,Pedalcycle
0,345979,51.570865,-0.231959,2019-01-04T21:22:00Z,1,0,1,0,0,0,...,0,0,0,0,Night,1,0,1,0,0
1,345980,51.603859,-0.187240,2019-01-04T23:33:00Z,1,0,1,0,0,0,...,1,0,0,0,Night,2,0,1,0,0
2,345981,51.512198,-0.153122,2019-01-04T22:15:00Z,1,0,0,0,0,0,...,0,0,1,0,Night,1,0,0,0,0
3,345982,51.431480,-0.016083,2019-01-04T18:00:00Z,1,0,0,0,0,0,...,0,0,0,0,Evening,1,0,1,0,0
4,345983,51.473487,0.145202,2019-01-04T20:45:00Z,1,0,0,1,0,0,...,1,0,0,0,Night,2,0,0,0,0


In [ ]:
df_merged.to_csv('../dataframes/merged_nov_29.csv', index=False)